목표 : 지하철역 근처의 아파트들의 시세를 시계열로 나타낸다

-역별로 근처 아파트를 그룹화한다  
-그룹화된 아파트들의 시세를 시계열로 나타낸다. 가능하면 그래프로 그려낸다  

###### 먼저 1호선을 기준으로 시행

In [3]:
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim

import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [4]:
stay = pd.read_csv('data/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding='cp949')
stay.index = stay['역명']
del stay['역명']
one = stay[stay['호선']==1]
del one['데이터기준일자']
one

,연번,호선,역번호,경도,위도
역명,,,,,
서울,1,1,150,126.972533,37.553150
시청,2,1,151,126.975407,37.563590
종각,3,1,152,126.983116,37.570203
종로3가,4,1,153,126.992095,37.570429
종로5가,5,1,154,127.001900,37.570971
동대문,6,1,155,127.007890,37.565145
동묘앞,7,1,159,127.016459,37.573265
신설동,8,1,156,127.024710,37.576117
제기동,9,1,157,127.034902,37.578116


In [32]:
apt = pd.read_csv('data/서울특별시_광진구_아파트정보_20210208.csv', encoding='cp949')
apt.drop(['관리사무소연락처', '관리사무소팩스', '데이터기준일자'], axis=1, inplace=True)
apt

,구분,단지명,아파트소재지,동수,세대수,층수
0,아파트,삼민,서울특별시 광진구 긴고랑로15길 32(중곡2동609),1,36,6
1,아파트,중곡2차,서울특별시 광진구 긴고랑로1길 55(중곡3동190-26),3,120,5
2,아파트,중곡1차,서울특별시 광진구 동일로72길 17(중곡3동191-77),3,150,5
3,아파트,중곡성원,서울특별시 광진구 동일로 459(중곡3동681),1,91,24
4,아파트,광덕,서울특별시 광진구 동일로76가길 17(중곡3동683),1,55,12
...,...,...,...,...,...,...
90,아파트,더샵스타시티,서울특별시 광진구 아차산로 262(자양3동 227-7),4,1177,58
91,아파트,이튼타워리버3차,서울특별시 광진구 능동로 18(자양3동 855),3,260,25
92,아파트,래미안구의파크스위트,서울특별시 광진구 광나루로 545(구의2동 668),12,854,23
93,아파트,테라팰리스건대1차,서울특별시 광진구 아차산로25길 60(화양동 531),1,54,7


In [41]:
import re
apt['아파트소재지'] = apt['아파트소재지'].str.replace(r'\([^)]*\)', '', regex=True)
apt

,구분,단지명,아파트소재지,동수,세대수,층수
0,아파트,삼민,서울특별시 광진구 긴고랑로15길 32,1,36,6
1,아파트,중곡2차,서울특별시 광진구 긴고랑로1길 55,3,120,5
2,아파트,중곡1차,서울특별시 광진구 동일로72길 17,3,150,5
3,아파트,중곡성원,서울특별시 광진구 동일로 459,1,91,24
4,아파트,광덕,서울특별시 광진구 동일로76가길 17,1,55,12
...,...,...,...,...,...,...
90,아파트,더샵스타시티,서울특별시 광진구 아차산로 262,4,1177,58
91,아파트,이튼타워리버3차,서울특별시 광진구 능동로 18,3,260,25
92,아파트,래미안구의파크스위트,서울특별시 광진구 광나루로 545,12,854,23
93,아파트,테라팰리스건대1차,서울특별시 광진구 아차산로25길 60,1,54,7


In [46]:
address = apt['아파트소재지']
address

0     서울특별시 광진구 긴고랑로15길 32
1      서울특별시 광진구 긴고랑로1길 55
2      서울특별시 광진구 동일로72길 17
3        서울특별시 광진구 동일로 459
4     서울특별시 광진구 동일로76가길 17
              ...         
90      서울특별시 광진구 아차산로 262
91        서울특별시 광진구 능동로 18
92      서울특별시 광진구 광나루로 545
93    서울특별시 광진구 아차산로25길 60
94    서울특별시 광진구 아차산로25길 60
Name: 아파트소재지, Length: 95, dtype: object

In [43]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

In [52]:
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    except:
        return[0,0]

In [53]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [55]:
print(latitude)
print(longitude)

[37.562648, 0, 37.5655463, 37.5378567, 37.5693387, 37.5585867, 37.5410325, 37.5538674, 37.54795875, 37.5554561, 37.5372439, 37.5372439, 37.53781, 37.53781, 37.5363797, 37.5384199, 37.5414127, 37.5372439, 37.5436477, 37.5415571, 37.5436477, 37.5436477, 37.5503428, 37.54107, 37.5436477, 37.5436477, 37.54107, 37.54107, 37.5436477, 37.5467113, 37.5436477, 37.5436477, 37.5428876, 37.5441483, 37.5439347, 37.5554561, 37.5436477, 37.5439347, 37.5436477, 37.5362328, 37.5395107, 37.5296862, 37.5372439, 37.5293201, 37.529709, 37.5283532, 37.5300792, 37.5298151, 37.5303492, 37.5279684, 37.5283532, 37.529709, 37.529916, 37.5370591, 37.5370591, 37.5316065, 37.5353033, 37.5370591, 37.5364687, 37.5338573, 37.5355421, 37.5378234, 37.5321631, 37.5321631, 37.5320251, 37.5308138, 37.5338573, 37.5335318, 37.5378234, 37.5308138, 37.5338573, 37.5320251, 37.5324771, 37.5320251, 37.5335692, 37.5378419, 0, 37.5324228, 37.5335692, 37.5347614, 37.5364436, 0, 37.5436173, 37.5455898, 37.5507517, 37.5372439, 37.5360

In [57]:
stay_df = pd.DataFrame({'위도':stay['위도'].values,'경도':stay['경도'].values})
address_df = pd.DataFrame({'위도':latitude,'경도':longitude})
address_df

,위도,경도
0,37.562648,127.083326
1,0.000000,0.000000
2,37.565546,127.079528
3,37.537857,127.062466
4,37.569339,127.080247
...,...,...
90,37.543648,127.102958
91,37.537842,127.068701
92,37.543935,127.092727
93,37.542256,127.066902


In [72]:
stay_df['위도'][1]

37.56359

In [76]:
from haversine import haversine

start = (float(stay_df['위도'][0]), float(stay_df['경도'][0]))
goal = (float(address_df['위도'][0]), float(address_df['경도'][0]))

print(start, goal)

haversine(start, goal, unit='m')

(37.55315, 126.972533) (37.562648, 127.0833258)


9823.162527738179

## 광진구 아파트단지와 서울 지하철역간 거리(km단위로 나타냄)

In [88]:
result = []

for i in range(len(stay_df)):
    start = (float(stay_df['위도'][i]), float(stay_df['경도'][i]))
    for j in range(len(address_df)):
        goal = (float(address_df['위도'][j]), float(address_df['경도'][j]))
        print(start, goal)
        geoly = haversine(start, goal)
        print(f'{geoly:.2f}km')
        print('--------------------------')
        result.append([start, goal, geoly])

(37.55315, 126.972533) (37.562648, 127.0833258)
9.82km
--------------------------
(37.55315, 126.972533) (0.0, 0.0)
13174.08km
--------------------------
(37.55315, 126.972533) (37.5655463, 127.0795285)
9.53km
--------------------------
(37.55315, 126.972533) (37.5378567, 127.0624661)
8.11km
--------------------------
(37.55315, 126.972533) (37.5693387, 127.0802473)
9.66km
--------------------------
(37.55315, 126.972533) (37.5585867, 127.0871514)
10.12km
--------------------------
(37.55315, 126.972533) (37.5410325, 127.086969)
10.18km
--------------------------
(37.55315, 126.972533) (37.5538674, 127.09686)
10.96km
--------------------------
(37.55315, 126.972533) (37.54795875, 127.09334624735006)
10.67km
--------------------------
(37.55315, 126.972533) (37.5554561, 127.0845439)
9.88km
--------------------------
(37.55315, 126.972533) (37.5372439, 127.0931747)
10.78km
--------------------------
(37.55315, 126.972533) (37.5372439, 127.0931747)
10.78km
--------------------------
(37.5

문제점1. address 중간에 좌표가 안잡히는 곳이 있음  
문제점2. 보기나쁨.

In [89]:
result

[[(37.55315, 126.972533), (37.562648, 127.0833258), 9.82316252773818],
 [(37.55315, 126.972533), (0.0, 0.0), 13174.076763059815],
 [(37.55315, 126.972533), (37.5655463, 127.0795285), 9.531509500117819],
 [(37.55315, 126.972533), (37.5378567, 127.0624661), 8.109102309340486],
 [(37.55315, 126.972533), (37.5693387, 127.0802473), 9.663570744488306],
 [(37.55315, 126.972533), (37.5585867, 127.0871514), 10.121788587927087],
 [(37.55315, 126.972533), (37.5410325, 127.086969), 10.178406787693321],
 [(37.55315, 126.972533), (37.5538674, 127.09686), 10.960178639788372],
 [(37.55315, 126.972533),
  (37.54795875, 127.09334624735006),
  10.666191419876107],
 [(37.55315, 126.972533), (37.5554561, 127.0845439), 9.877402220986383],
 [(37.55315, 126.972533), (37.5372439, 127.0931747), 10.782254341161043],
 [(37.55315, 126.972533), (37.5372439, 127.0931747), 10.782254341161043],
 [(37.55315, 126.972533), (37.53781, 127.09669), 11.078186463120913],
 [(37.55315, 126.972533), (37.53781, 127.09669), 11.078